##  Box 스마트 포인터 


- 컴파일 시간에 크기를 알 수 없는 유형이 있고 정확한 크기가 필요한 컨텍스트에서 해당 유형의 값을 사용하려는 경우
- 많은 양의 데이터가 있고 소유권을 이전하고 싶지만 그렇게 할 때 데이터가 복사되지 않도록 할 때
- 값을 소유하고 싶고 특정 유형이 아닌 특정 특성을 구현하는 유형이라는 점에만 관심이 있는 경우

## 1. 동적타입 : Dynamically-Sized Types

- Box 타입의 주요 이점 중 하나는 동적 크기 타입(DST)의 값을 힙에 저장할 수 있다는 점입니다. 
- DST는 str 또는 [T]와 같이 컴파일 시점에 크기를 알 수 없는 타입입니다. 컴파일 시점에 DST의 크기를 알 수 없으므로 스택에 저장할 수 없습니다. 
- 대신 Box와 같은 스마트 포인터를 사용하여 힙에 저장해야 합니다.


In [2]:
fn main() {
    let s = Box::new("hello");
    println!("s = {}", s);
}

In [3]:
main()

s = hello


()

## 2. 소유권과 빌림

- 박스 타입의 또 다른 장점은 변수와 함수 간에 값의 소유권을 이전할 수 있다는 것입니다. 
- Rust에서 소유권은 누가 언제 메모리를 할당 해제할 책임이 있는지 결정하는 핵심 개념입니다. 
- Box 타입을 사용하면 Box 인스턴스를 변수와 함수 간에 이동하여 Box 인스턴스에 저장된 값의 소유권을 이전할 수 있습니다.


### 소유권 이동 후에 변수 사용하면 에러

In [4]:
fn main() {
    let a = Box::new(5);
    let b = a;
    println!("a = {}", a); // error: use of moved value
    println!("b = {}", b);
}

Error: borrow of moved value: `a`

### 소유권이동 후에는 사용하지 않는다

In [20]:
fn main() {
    let a = Box::new(5);
    let b = a;
    //println!("a = {}", a); // error: use of moved value
    println!("b = {}", b);
}

In [21]:
main()

b = 5


()

## 3. When to Use Box

- 스택의 공간을 절약하기 위해 힙에 저장하려는 큰 값이 있습니다.
- 현재 함수의 수명을 초과하는 수명을 가진 값이 있으며 현재 범위 밖으로 '이동'해야 하는 경우.
- 동적 디스패치가 필요하고 힙에 저장해야 하는 특성 객체 유형의 값이 있습니다.
- 링크된 목록과 같은 재귀적 데이터 구조가 있어 스택 오버플로를 유발할 수 있습니다.

## 3-1 스택에서 힙으로 저장, 참조, 역참조 사용

In [6]:
fn main() {
    let x = Box::new(5);
    let y = &x;
    let z = *x + 100; // 정수값  5 + 100
    
    println!("x={}, y={}, z={}", x, y,z);
}

In [7]:
main()

x=5, y=5, z=105


()

### 역참조 함수를 사용해서 처리 

In [8]:
fn main() {
    let x = Box::new(5);
    let y = std::ops::Deref::deref(&x);
    println!("{}", y);
}

In [9]:
main()

5


()

### 복제해서 처리하기 

In [10]:
fn main() {
    let x = Box::new(5);
    let y = x.clone();
    println!("{:?} = {:?}", x, y);
}

In [11]:
main()

5 = 5


()

## 3-2 Box를 사용하여 힙의 데이터 가리키기

- Box의 가장 일반적인 용도 중 하나는 힙에 있는 데이터를 가리키는 포인터를 만드는 것입니다. 
- 힙의 데이터를 가리키는 Box를 생성할 때는 Box가 범위를 벗어날 때 데이터가 올바르게 할당 해제되도록 해야 합니다.

In [12]:
fn main() {
    let b = Box::new("hello".to_string());
    println!("{}", b);
}

In [13]:
main()

hello


()

## 3-3 Box를 사용하여 재귀적 데이터 구조에 데이터 저장하기

- Box의 또 다른 일반적인 용도는 링크된 목록이나 이진 트리와 같은 재귀적 데이터 구조에 데이터를 저장하는 것입니다. 
- 재귀 구조의 데이터 크기는 컴파일 시점에 알 수 없으므로 구조에 데이터를 직접 포함하지 않고 포인터를 사용하여 힙에 저장해야 합니다.

In [43]:
#[derive(Debug)]
enum List {
    Cons(i32, Box<List>),
    Nil,
}

impl List {
        // Return representation of the list as a (heap allocated) string
    fn stringify(&self) -> String {
        match *self {
            Cons(head, ref tail) => {
                format!("{}, {}", head, tail.stringify())
            },
            Nil => {
                format!("Nil")
            },
        }
    }

}


fn main() {
    let list = Cons(1, Box::new(Cons(2, Box::new(Cons(3, Box::new(Nil))))));
    println!("{:?}", list);
    let ll = match list {
            Cons(head, ref tail) => {
                // `format!` is similar to `print!`, but returns a heap
                // allocated string instead of printing to the console
                format!("{}, {}", head, tail.stringify())
            },
            Nil => {
                format!("Nil")
            },
        };
    println!("ll {:?}", ll);
}

In [44]:
main()

Cons(1, Cons(2, Cons(3, Nil)))
ll "1, 2, 3, Nil"


()

## 3-4 Box를 사용하여 벡터에 데이터 저장하기
- Box를 사용하여 데이터를 벡터에 저장하여 서로 다른 유형의 데이터를 동일한 벡터에 저장할 수도 있습니다. 
- 이 기능은 이질적인 데이터 컬렉션을 저장하고 싶지만 Vec<Any>와 같은 더 복잡한 데이터 구조를 사용하고 싶지 않을 때 유용합니다.

In [18]:
fn main() {
    let v: Vec<Box<dyn std::fmt::Debug>> = vec![
        Box::new(1),
        Box::new("hello"),
        Box::new(vec![1, 2, 3]),
    ];

    for item in &v {
        println!("{:?}", item);
    }
}

In [19]:
main()

1
"hello"
[1, 2, 3]


()

## 링크드 리스트 구현

In [32]:
#[derive(Debug)]
enum List {
    // Cons: Tuple struct that wraps an element and a pointer to the next node
    Cons(u32, Box<List>),
    // Nil: A node that signifies the end of the linked list
    Nil,
}

// Methods can be attached to an enum
impl List {
    // Create an empty list
    fn new() -> List {
        // `Nil` has type `List`
        Nil
    }

    // Consume a list, and return the same list with a new element at its front
    fn prepend(self, elem: u32) -> List {
        // `Cons` also has type List
        Cons(elem, Box::new(self))
    }

    // Return the length of the list
    fn len(&self) -> u32 {
        // `self` has to be matched, because the behavior of this method
        // depends on the variant of `self`
        // `self` has type `&List`, and `*self` has type `List`, matching on a
        // concrete type `T` is preferred over a match on a reference `&T`
        match *self {
            // Can't take ownership of the tail, because `self` is borrowed;
            // instead take a reference to the tail
            Cons(_, ref tail) => 1 + tail.len(),
            // Base Case: An empty list has zero length
            Nil => 0
        }
    }

    // Return representation of the list as a (heap allocated) string
    fn stringify(&self) -> String {
        match *self {
            Cons(head, ref tail) => {
                // `format!` is similar to `print!`, but returns a heap
                // allocated string instead of printing to the console
                format!("{}, {}", head, tail.stringify())
            },
            Nil => {
                format!("Nil")
            },
        }
    }
}

In [33]:
fn main() {
    // Create an empty linked list
    let mut list = List::new();

    // Prepend some elements
    list = list.prepend(1);
    list = list.prepend(2);
    list = list.prepend(3);

    // Show the final state of the list
    println!("linked list has length: {}", list.len());
    println!("{}", list.stringify());
}

In [34]:
main()

linked list has length: 3
3, 2, 1, Nil


()